# Table of Contents
* [Import & Path](#Import-&-Path)
* [Dataset Preparation](#Dataset-Preparation)
  * [General Configuration](#General-Configuration)
    * [Get Classes](#Get-Classes)
  * [Collecting Training Data](#Collecting-Training-Data)
    * [Shuffling Training Data](#Shuffling-Training-Data)
    * [Train Validation Split](#Train-Validation-Split)
    * [Encoding the Labels](#Encoding-the-Labels)
  * [Set Configuration Variable And Training Data](#Set-Configuration-Variable-And-Training-Data)
* [Create and Train Models](#Create-and-Train-Models)
    * [Create Model](#Create-Model)
        * [Evaluate Model](#Evaluate-Model)
    * [Create Pretrain Models](#Create-Pretrain-Models)
    * [Train Model](#Train-Model)
        * [Train with 64 Batch](#Train-with-64-Batch)
            * [0.001 Learning Rate](#0.001-Learning-Rate)
            * [0.0001 Learning Rate](#0.0001-Learning-Rate)
        * [Train with 32 Batch](#Train-with-32-Batch)
            * [0.001 Learning Rate](#0.001-Learning-Rate)
            * [0.0001 Learning Rate](#0.0001-Learning-Rate)
        * [Train with 16 Batch](#Train-with-16-Batch)
            * [0.001 Learning Rate](#0.001-Learning-Rate)
            * [0.0001 Learning Rate](#0.0001-Learning-Rate)
        * [Train with 8 Batch](#Train-with-8-Batch)
            * [0.001 Learning Rate](#0.001-Learning-Rate)
            * [0.0001 Learning Rate](#0.0001-Learning-Rate)
* [Score](#Score)
    * [Load Model List and Evaluate Data](#Load-Model-List-and-Evaluate-Data)
        * [Show Model List](#Show-Model-List)
    * [Load Test Data](#Load-Test-Data)
    * [Create Score Functions](#Create-Score-Functions)
    * [Predict Test Data](#Predict-Test-Data)
    * [Show Testing Sample](#Show-Testing-Sample)

# Import & Path

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, Sequential
from keras.layers import Input, Activation, Dense, Flatten
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from PIL import Image
from PIL import ImageFilter
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import os
import math
import pickle

np.random.seed(42)

In [ ]:
data_dir = os.getcwd()
trainPath = os.path.join(os.getcwd(), "Path/to/TrainImages")
testPath = os.path.join(os.getcwd(), "Path/to/TestImages")
testPath2 = os.path.join(os.getcwd(), "Online-Test-sort")
modelPath = os.path.join(os.getcwd(), "model")
print(trainPath)
print(testPath)

# Dataset Preparation

## General Configuration
Set global variable with general conf. The conf mention below:

| Image size   |    | Color mode   |    | Epochs |
|--------------|----|--------------|----|--------|
| 75 x 75 pixel|    |RGB/3 Channels|    | 20     |

In [ ]:
height = 75
width = 75
channels = 3
epochs = 20

### Get Classes
Set total classes and classes label

In [ ]:
NUM_CATEGORIES = len(os.listdir(trainPath))
NUM_CATEGORIES

In [ ]:
# Label Overview
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

## Collecting Training Data
Collecting and processing training data

In [ ]:
image_data = []
image_labels = []

trainDir = os.listdir(trainPath)
i = 0
for dir in trainDir:
    path = os.path.join(trainPath, dir)
    images = os.listdir(path)

    for img in images:
        if not img.endswith('.jpg'):
            continue
        else:
            try:
                image = Image.open(path + '/' +img)
                resize_image = image.resize((height, width))
                enchanced_image = resize_image.filter(ImageFilter.SHARPEN)
                final_image = enchanced_image#.convert('L').convert('RGB')
                image_data.append(np.array(final_image))

                image_labels.append(i)
            except:
                print("Error in " + img)
                
    i = i + 1

# Changing the list to numpy array
image_data = np.array(image_data)
image_labels = np.array(image_labels)

print(image_data.shape, image_labels.shape)

### Shuffling Training Data

In [ ]:
shuffle_indexes = np.arange(image_data.shape[0])
np.random.shuffle(shuffle_indexes)
image_data = image_data[shuffle_indexes]
image_labels = image_labels[shuffle_indexes]

### Train Validation Split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(image_data, image_labels, test_size=0.1, random_state=42, shuffle=True)

X_train = X_train/255 
X_val = X_val/255

print("X_train.shape", X_train.shape)
print("X_valid.shape", X_val.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_val.shape)

### Encoding the Labels

In [ ]:
y_train = keras.utils.to_categorical(y_train, NUM_CATEGORIES)
y_val = keras.utils.to_categorical(y_val, NUM_CATEGORIES)

print(y_train.shape)
print(y_val.shape)

## Set Configuration Variable And Training Data
Set configuration and train valid test datasets with tensorflow

Training dataset with several configurations as mention below:

 - 4 Architecture, 3 Batch Size, 2 Learning Rate

| Architecture |    | Batch Size |    | Learning Rate |
|--------------|----|------------|----|---------------|
| VGG-16       |    | 8          |    | 0.001         |
| ResNet-50    |    | 16         |    | 0.0001        |
| Inception-V3 |    | 32         |
| Xception     |    | 64         |

# Create and Train Models

## Create Model
Create global model function

In [ ]:
def TrainModel(pretrain_model, batch_size, learning_rate, name):
    aug = ImageDataGenerator()

    model = Sequential()
    model.add(pretrain_model)
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(43, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=math.pow(10, (learning_rate * -1))), metrics=['accuracy'])
    history = model.fit(aug.flow(X_train, y_train, batch_size=batch_size), epochs=epochs, validation_data=(X_val, y_val))
    model.save('model/' + name + '.h5')

    return history

### Evaluate Model
Create global model evaluation function

In [ ]:
def EvaluateModel(name, history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs_range = range(epochs)

    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.savefig('model/' + name + '.png')
    plt.show()

## Create Pretrain Models

In [ ]:
pretrain_vgg16 = tf.keras.applications.vgg16.VGG16(
    include_top = False,
    input_shape = (height, width, channels),
    pooling = 'avg',
    classes = 43,
    weights = 'imagenet'
)

pretrain_vgg16.trainable = False

In [ ]:
pretrain_resnet50 = tf.keras.applications.ResNet50(
    include_top = False,
    input_shape = (height, width, channels),
    pooling = 'avg',
    classes = 43,
    weights = 'imagenet'
)

pretrain_resnet50.trainable = False

In [ ]:
pretrain_inceptionv3 = tf.keras.applications.inception_v3.InceptionV3(
    include_top = False,
    input_shape = (height, width, channels),
    pooling = 'avg',
    classes = 43,
    weights = 'imagenet'
)

pretrain_inceptionv3.trainable = False

In [ ]:
pretrain_xception = tf.keras.applications.xception.Xception(
    include_top = False,
    input_shape = (height, width, channels),
    pooling = 'avg',
    classes = 43,
    weights = 'imagenet'
)

pretrain_xception.trainable = False

## Train Model

In [ ]:
def TrainProcess(pretrain_model, batch_size, learning_rate, model_name):
    name = '{}-{}-{}'.format(model_name, batch_size, learning_rate)
    history = TrainModel(pretrain_model, batch_size, learning_rate, name)

    with open('./model/' + name + '.pickle', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    EvaluateModel(name, history)

In [ ]:
def Train(batch_size, learning_rate):
    TrainProcess(pretrain_vgg16, batch_size, learning_rate, 'vgg16')
    TrainProcess(pretrain_resnet50, batch_size, learning_rate, 'resnet50')
    TrainProcess(pretrain_inceptionv3, batch_size, learning_rate, 'inceptionv3')
    TrainProcess(pretrain_xception, batch_size, learning_rate, 'xception')

### Train with 64 Batch

#### 0.001 Learning Rate

In [ ]:
batch_size = 64
learning_rate = 3

Train(batch_size, learning_rate)

#### 0.0001 Learning Rate

In [ ]:
batch_size = 64
learning_rate = 4

Train(batch_size, learning_rate)

### Train with 32 Batch

#### 0.001 Learning Rate

In [ ]:
batch_size = 32
learning_rate = 3

Train(batch_size, learning_rate)

#### 0.0001 Learning Rate

In [ ]:
batch_size = 32
learning_rate = 4

Train(batch_size, learning_rate)

### Train with 16 Batch

#### 0.001 Learning Rate

In [ ]:
batch_size = 16
learning_rate = 3

Train(batch_size, learning_rate)

#### 0.0001 Learning Rate

In [ ]:
batch_size = 16
learning_rate = 4

Train(batch_size, learning_rate)

### Train with 8 Batch

#### 0.001 Learning Rate

In [ ]:
batch_size = 8
learning_rate = 3

Train(batch_size, learning_rate)

#### 0.0001 Learning Rate

In [ ]:
batch_size = 8
learning_rate = 4

Train(batch_size, learning_rate)

# Score

## Load Model List and Evaluate Data

In [ ]:
modelList = pd.DataFrame(columns=['model', 'accuracy', 'loss', 'val_accuracy', 'val_loss'])

for x in os.listdir('./model'):
    if x.endswith('.h5'):
        modelList = modelList.append({'model': x.split('.')[0]}, ignore_index=True)
    elif x.endswith('.pickle'):
        with open('./model/' + x, 'rb') as file_pi:
            history = pickle.load(file_pi)
            modelList.loc[modelList['model'] == x.replace('.pickle', '.h5'), 'accuracy'] = history['accuracy'][-1]
            modelList.loc[modelList['model'] == x.replace('.pickle', '.h5'), 'loss'] = history['loss'][-1]
            modelList.loc[modelList['model'] == x.replace('.pickle', '.h5'), 'val_accuracy'] = history['val_accuracy'][-1]
            modelList.loc[modelList['model'] == x.replace('.pickle', '.h5'), 'val_loss'] = history['val_loss'][-1]

### Show Model List
Show model list and save to excel

In [ ]:
for x in modelList['model']:
    print(x)

print(len(modelList))

modelList.to_excel("modelList.xlsx", index = False)

## Load Test Data

In [ ]:
test = pd.read_csv(data_dir + '/Test.csv')

labels = test["ClassId"].values
imgs = test["Path"].values

data = []

for img in imgs:
    try:
        image = Image.open(data_dir + '/' +img)
        resize_image = image.resize((height, width))
        enchanced_image = resize_image.filter(ImageFilter.SHARPEN)
        data.append(np.array(resize_image))
    except:
        print("Error in " + img)
X_test = np.array(data)
X_test = X_test/255

## Create Score Functions
Create confusion matrix and classification report

In [ ]:
def confMatrix(modelName, labels, pred):
    cf = confusion_matrix(labels, pred)
    plt.clf()
    plt.figure(figsize = (20,20))
    sns.heatmap(cf, annot=True)
    plt.savefig('model/' + modelName + '-confusion-matrix.png')

In [ ]:
def classificationReport(modelName, labels, pred):
    cr = classification_report(labels, pred, output_dict=True)

    df = pd.DataFrame(cr).transpose()
    df.to_excel('model/' + modelName + '-classification-report.xlsx')

## Predict Test Data
Load model and predict test data, then save to excel

In [ ]:
max_accuracy = 0
max_accuracy_model = ''
max_accuracy_pred = []

for x in modelList['model']:
    model = keras.models.load_model('model/' + x + '.h5')
    
    predict_x = model.predict(X_test)
    pred = np.argmax(predict_x, axis=1)

    accuracy = accuracy_score(labels, pred)
    print('Test Data accuracy: ', accuracy * 100)

    confMatrix(x, labels, pred)
    classificationReport(x, labels, pred)

    if accuracy > max_accuracy:
        max_accuracy = accuracy
        max_accuracy_model = x
        max_accuracy_pred = pred

print('Max accuracy: ', max_accuracy * 100)
print('Max accuracy model: ', max_accuracy_model)

## Show Testing Sample
Show testing sample from max accuracy model

In [ ]:
plt.figure(figsize = (25, 25))

start_index = 0
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    prediction = max_accuracy_pred[start_index + i]
    actual = labels[start_index + i]
    col = 'g'
    if prediction != actual:
        col = 'r'
    plt.xlabel('Actual={} || Pred={}'.format(actual, prediction), color = col)
    plt.imshow(X_test[start_index + i])
plt.show()